# AG NEWS 모델 만들기
- 한국어 어떻게 처리할지 생각하기

-> https://tutorials.pytorch.kr/beginner/text_sentiment_ngrams_tutorial.html

In [46]:
import torch
from torchtext.datasets import AG_NEWS

# data pipe타입 > iterator 타입 형변환
train_iter = iter(AG_NEWS(split='train'))

반복자 왜? -> raw data에 접근하기 위해서 

In [47]:
# 데이터 확인 => (label, text), label 1~4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

# 2. 데이터 처리 파이프라인 준비
- 어휘집 vocab, 단어 벡터word vector, 토크나이저 tokenizer
- 가공되지 않은 문자열에 대한 데이터 처리 빌딩 블록
- 일반적인 NLP 데이터 처리
    - 첫번째 단계 : 가공되지 않은 학습 데이터셋으로 어휘집 생성
        => 토큰 목록 또는 반복자 받는 내장 팩토리 함수(factory function): build_vocab_from_iterator
    - 사용자는 어휘집에 추가할 특수 기호(special symbol) 전달 가능

In [48]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# 특별문자토큰
UNK = "<UNK>"
PAD = "<PAD>"

# 토커나이저 생성
tokenizer = get_tokenizer("basic_english")

# 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split='train')

In [49]:
# 한글 ver
from konlpy.tag import Okt

# tokenizer = Okt()

# 밑에서 yield tokenizer.morphs(text)

In [50]:
# 토큰 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트가 나옴 -> 텍스트만 토큰화하면 됨 
        yield tokenizer(text)

In [51]:
# 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

# unk 인덱스 0으로 설정
vocab.set_default_index(vocab["<unk>"])

In [52]:
vocab(["<unk>", "here", "is", "an", "example"])

[0, 475, 21, 30, 5297]

In [53]:
# 텍스트 -> 정수 인코딩
text_pipeline = lambda x: vocab(tokenizer(x)) # 다 쪼갠게 들어감

# 레이블 -> 정수 인코딩
label_pipeline = lambda x: int(x) -1 # 레이블=1~4를 0~3으로 만드는 것

# 3. 데이터 배치와 반복자 생성
- torch.utils.data.DataLoader : getitem(), len() 구현한 맵 형태(map-style)
- collate_fn(): DataLoader로부터 생성된 샘플 배치 함수
    - 입력 : DataLoader에 배치 크가가 있는 배치 데이터

In [54]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 배치 크기 만큼 데이터셋 반환 함수
def collate_batch(batch):
    # 배치크기 만큼의 라벨, 텍스트, 오프셋 값 저장 변수
    label_list, text_list, offsets = [], [], [0] # offset은 글자마다 길이가 다르니까 해당 글자의 길이에 대한 정보를 주는 것
    
    # 1개씩 뉴스기사, 라벨 추출해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))
        
        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        
        # 텍스트 offset 즉, 텍스트 크기/길이 저장 
        offsets.append(processed_text.size(0))
    
    # 텐서화
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [55]:
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, 
                        batch_size=8,
                        shuffle=False, 
                        collate_fn=collate_batch)

In [56]:
# 분류 클래스 수와 단어 사전 개수
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f"num_class : {num_class}\nvocab_size : {vocab_size}")

num_class : 4
vocab_size : 95811


In [57]:
type(vocab)

torchtext.vocab.vocab.Vocab

In [58]:
for labels, texts, offsets in dataloader:
    print(labels, texts, offsets) 
    break

tensor([2, 2, 2, 2, 2, 2, 2, 2]) tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
           14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
            8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
          294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
            1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
           27,    14,    27,   

# 모델 정의하기
- nn.EmbeddingBag -> 레이어 / 분류를 위한 선형 레이어
- nn.EmbeddingBag -> 모듈은 텍스의 길이를 오프셋으로 저장하고 있어 padding 필요 없음 

In [59]:
from torch import nn
# 입력층 : EmbeddingBag Layer - 레이어와 분류 목적을 위한 선형 레이어, 텍스트의 길이는 offset
# 은닉층 : Linear - 4개 클래스 분류

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        
        self.init_weights() # 가중치 초기화, 클래스 내의 함수 호출
    
    # 가중치 초기화
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange) # uniform : 균등분포, -0.5~0.5 범위로 
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
    
    # 순방향 학습 진행
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

# 인스턴스 생성하기

In [60]:
# 학습 관련 하이퍼파라미터와 인스턴스
# HIDDEN_SIZE = 3
# EMBEDDING_DIM = 64
# VOCAB_SIZE = len(vocab)
# NUM_CLASS = len(set([label for label, _ in train_iter]))
# EPOCHS = 10
# LR = 5
# BATCH_SIZE = 64
#################################위에는 강사님 밑에는 예제 코드

emsize = 64

# vocab_size-> 95811, embed_dim -> 64, num_class -> 4
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

# 모델을 학습하고 결과를 평가하는 함수 정의하기

In [61]:
import time

def train(dataloader):
    model.train()
    
    # 학습 평가 관련 변수들
    total_acc, total_count = 0, 0
    log_interval = 300
    start_time = time.time()
    
    # 배치 학습 진행
    for idx, (label, text, offsets) in enumerate(dataloader):
        
        label, text, offsets = label.to(device), text.to(device), offsets.to(device)
        
        # 학습 진행 
        predicted_label = model(text, offsets)
        
        # 손실 계산 및 W, b업데이트 
        optimizer.zero_grad()
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # 기울기 폭주/소실 방지를 위해 양극단 컷 
        optimizer.step()
        
        # 배치 학습 평가 
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [62]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# 맵(map)처럼 인덱스/키로 데이터 샘플을 얻어옴

# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR) # 최적화 함수
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1) # 스케줄러
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)

# train set을 train과 validation으로 분리
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   300/ 1782 batches | accuracy    0.601
| epoch   1 |   600/ 1782 batches | accuracy    0.819
| epoch   1 |   900/ 1782 batches | accuracy    0.856
| epoch   1 |  1200/ 1782 batches | accuracy    0.871
| epoch   1 |  1500/ 1782 batches | accuracy    0.880
-----------------------------------------------------------
| end of epoch   1 | time: 17.60s | valid accuracy    0.890 
-----------------------------------------------------------
| epoch   2 |   300/ 1782 batches | accuracy    0.898
| epoch   2 |   600/ 1782 batches | accuracy    0.901
| epoch   2 |   900/ 1782 batches | accuracy    0.900
| epoch   2 |  1200/ 1782 batches | accuracy    0.903
| epoch   2 |  1500/ 1782 batches | accuracy    0.903
-----------------------------------------------------------
| end of epoch   2 | time: 17.32s | valid accuracy    0.898 
-----------------------------------------------------------
| epoch   3 |   300/ 1782 batches | accuracy    0.915
| epoch   3 |   600/ 1782 batches | accuracy

In [63]:
print(type(train_dataset))
print()
print(train_dataset[0])

<class 'torchtext.data.functional.to_map_style_dataset.<locals>._MapStyleDataset'>

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")


In [64]:
def predict(model, text, text_pipeline):
    with torch.no_grad():
        # 토큰화 > 정수 변환 > 텐서
        text = torch.tensor(text_pipeline(text)) # text_pipeline = lambda x: vocab(tokenizer(x))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1 # 라벨이 1 2 3 4라서 